In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'train'

In [4]:
# create model structure CNN
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])


C:\Users\User\Downloads\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50


C:\Users\User\AppData\Local\Temp/ipykernel_18444/3198193097.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 118s 262ms/step - loss: 1.6284 - accuracy: 0.3649 - val_loss: 1.5444 - val_accuracy: 0.4184
Epoch 2/50
448/448 [==============================] - 114s 254ms/step - loss: 1.5328 - accuracy: 0.4075 - val_loss: 1.4642 - val_accuracy: 0.4443
Epoch 3/50
448/448 [==============================] - 114s 255ms/step - loss: 1.4625 - accuracy: 0.4378 - val_loss: 1.3995 - val_accuracy: 0.4727
Epoch 4/50
448/448 [==============================] - 117s 260ms/step - loss: 1.4013 - accuracy: 0.4654 - val_loss: 1.3536 - val_accuracy: 0.4867
Epoch 5/50
448/448 [==============================] - 98s 218ms/step - loss: 1.3510 - accuracy: 0.4882 - val_loss: 1.3199 - val_accuracy: 0.5054
Epoch 6/50
448/448 [==============================] - 91s 203ms/step - loss: 1.3091 - accuracy: 0.5043 - val_loss: 1.2781 - val_accuracy: 0.5193
Epoch 7/50
448/448 [==============================] - 90s 202ms/step - loss: 1.2680 - accuracy: 0.5212 - val_loss: 1.2494 - val_accuracy:

In [9]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)


In [10]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')
